# Quickstart tutorial: Enter the _Vivarium_

First follow the installations instruction in the README of the repository. If everything works correctly, you should have launched the web interface.

## Introduction to the web interface

For the web interface tutorial, please refer to [this file.](web_interface_tutorial.md)

## Controlling the simulator programmatically

Instead of controlling the simulator from the web interface, you can also control it programmatically, e.g. directly from this Jupyter notebook. First make sure that the simulator is stopped in the web interface. Then execute:

In [1]:
from vivarium.controllers.notebook_controller import NotebookController
controller = NotebookController()

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


The newly created `controller` object provides all the functionalities to easily interact with the simulator server. Let's look for example at the agents present in the environment:

In [2]:
controller.agents

This is a list of `Agent` objects, one per agent in the environment. Let's for example access the configuration of the first agent in this list and look at its attribute using the Python `.__dict__` syntax:

In [3]:
controller.agents[0].__dict__

{'config': AgentConfig(behavior=array([5, 5]), color='#0000ff', diameter=5.0, ent_sensedtype=0, exists=True, friction=0.10000000149011612, idx=0, left_motor=0.5, left_prox=0.0, mass_center=1.0, mass_orientation=0.125, max_speed=10.0, name='AgentConfig00004', orientation=0.7409666180610657, params=array([[[-1.,  0.,  1.],
         [ 0., -1.,  1.]],
 
        [[ 1.,  0.,  0.],
         [ 0.,  1.,  0.]]]), proximity_map_dist=array([ 0.        , 22.60018158, 38.96023941, 46.96992111, 53.13747406,
        57.41152191, 66.15736389, 13.83346939, 46.45539093, 56.80420685,
        46.81237793, 42.1365509 , 59.24790192, 25.2713623 , 48.95448303,
        58.85480499, 19.56998062, 56.59965515, 45.70919418, 34.97370148]), proximity_map_theta=array([ 0.        , -2.26088333, -2.57543445, -2.61952662, -1.2133019 ,
        -1.75308871,  1.57839966, -3.24084425,  1.0266974 , -3.25050664,
        -0.48111835, -2.70793176, -1.38904524, -1.38791978, -2.30412626,
         1.43002391, -2.99895096, -1.696334

We see the value of different attributes of the agent, for example its position, diameter or color. Let's make the first agent larger by increasing its `diameter` attribute:

In [4]:
controller.agents[0].diameter = 10.

If you now look at the web interface, you should see that one of the agent is now twice larger than the others. All changes made in this notebook will be automatically visible in the web interface, and vice versa. One can for example also change the color of an agent, let's do it on the third agent in the list now (i.e. the agent with index 2):

In [5]:
controller.agents[2].color = 'yellow'

The same principles apply to objects, which are accessible through `controller.objects`.

### Defining agent's behaviors

We can define the behavior of an agent as a Python function taking as argument an `Agent` and returning the values of the left and right motors, in this order. Within the body of the function, one can access all the agents attribute. Usually, defining a behavior requires to access the value of the agent sensors. This done through the `agent.left_prox` and `agent.right_prox` attributes, that return the value of the left and right sensors, respectively.

Let's define four behaviors, here corresponding to the four canonical behaviors of [Braitenberg Vehicles](https://en.wikipedia.org/wiki/Braitenberg_vehicle):

In [6]:
def aggression(agent):
    return agent.right_prox, agent.left_prox

def fear(agent):
    return agent.left_prox, agent.right_prox

def love(agent):
    return 1. - agent.left_prox, 1. - agent.right_prox

def shy(agent):
    return 1. - agent.right_prox, 1. - agent.left_prox

The code above only declares the behaviors, now we need to attach them to the agents in the simulator. Let's attach the `shy` behavior to the first five agents and set their color to blue ; and the `aggression` behavior to the four last agents and set their color to red:

In [7]:
for ag in controller.agents[:5]:
    ag.color = 'blue'
    ag.detach_all_behaviors()
    ag.attach_behavior(shy)
for ag in controller.agents[5:]:
    ag.color = 'red'
    ag.detach_all_behaviors()
    ag.attach_behavior(aggression)

All agents are now equipped with a behavior. We can launch the simulation with the code below ; then stop it with the code on the next line.

In [8]:
controller.run(threaded=True)

In [9]:
controller.stop()